# Customer.IO API Client Setup and Configuration

This notebook demonstrates how to set up and configure the Customer.IO API client library. It includes authentication testing, basic connectivity verification, and configuration examples for both US and EU regions.

In [ ]:
# Setup and Imports
import os
from datetime import datetime
from utils.api_client import CustomerIOClient
from utils.exceptions import CustomerIOError, ValidationError

print("Customer.IO API client library imported successfully")

# Configuration

Configure your Customer.IO API credentials. For security, credentials should be stored as environment variables rather than hardcoded in notebooks.

In [ ]:
# Configuration - Replace with your actual credentials
# Recommended: Set these as environment variables
API_KEY = os.getenv('CUSTOMERIO_API_KEY', 'your-api-key-here')
REGION = os.getenv('CUSTOMERIO_REGION', 'us')  # 'us' or 'eu'

print(f"Configured for region: {REGION}")
print(f"API key configured: {'Yes' if API_KEY != 'your-api-key-here' else 'No - using placeholder'}")

# Client Initialization

Initialize the Customer.IO client with your credentials and region settings.

In [ ]:
# Initialize the Customer.IO client
client = CustomerIOClient(
    api_key=API_KEY,
    region=REGION
)

print(f"Client initialized successfully")
print(f"Base URL: {client.base_url}")
print(f"Rate limit: {client.rate_limit_requests} requests per {client.rate_limit_window} seconds")

# Next Steps

Now that your client is configured and tested, you can proceed to explore specific API functionality:

- **01_people_management.ipynb** - User identification, deletion, and suppression
- **02_event_tracking.ipynb** - Custom and semantic event tracking  
- **03_objects_and_relationships.ipynb** - Object and relationship management
- **04_device_management.ipynb** - Device registration and management
- **05_batch_operations.ipynb** - Batch processing for high-volume operations

## Troubleshooting

**Authentication Errors:**
- Verify your API key is correct
- Ensure you're using the correct region (US vs EU)
- Check that your API key has the necessary permissions

**Rate Limiting:**
- The client automatically handles rate limiting (3000 requests per 3 seconds)
- For high-volume operations, use batch processing

In [ ]:
# Test user data for connectivity verification
test_user_id = f"test_user_{int(datetime.now().timestamp())}"
test_traits = {
    "email": "test@example.com",
    "name": "Test User",
    "test_setup": True
}

# Attempt to identify a test user
try:
    response = client.identify(user_id=test_user_id, traits=test_traits)
    print("✅ Connectivity test successful!")
    print(f"Response: {response}")
except CustomerIOError as e:
    print(f"❌ API Error: {e}")
    print("Check your API key and region settings")
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    print("Check your internet connection and configuration")

# Basic Connectivity Test

Test basic connectivity with a simple identify operation. This will verify that your credentials are working correctly.